# models

> Model and architecture tooling

In [ ]:
#|default_exp models

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

## Imports

In [ ]:
#| export
from AIsaac.utils import *
from AIsaac.dataloaders import *

import matplotlib.pyplot as plt,matplotlib as mpl
import torchvision.transforms.functional as TF,torch.nn.functional as F
import fastcore.all as fc
import torch
from torch import nn, Tensor
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import pandas as pd, numpy as np
import timm


In [ ]:
#|hide
import logging

## Setup

In [ ]:
#| hide
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
logging.disable(logging.WARNING)
set_seed(42)

## Load Data

In [ ]:
xmean,xstd = 0.28, 0.35

@inplace
def transformi(b): b['image'] = [(TF.to_tensor(o)-xmean)/xstd for o in b['image']]

_dataset = load_dataset('fashion_mnist').with_transform(transformi)
_dataset = sample_dataset_dict(_dataset)
dls = DataLoaders.from_dataset_dict(_dataset, 64, num_workers=4)
xb = fc.first(dls.train)[0]

  0%|          | 0/2 [00:00<?, ?it/s]

## Models

In [ ]:
#| export
@fc.delegates(timm.create_model)
def get_model_timm(model_name,**kwargs): 
    '''Loads model from timm, see timm.list_models for options'''
    return timm.create_model(model_name,**kwargs)

In [ ]:
model = get_model_timm('resnet18', pretrained=True,num_classes=10,in_chans=1).to(def_device)
fc.test_eq(model(fc.first(dls.train)[0].to(def_device)).shape,(64,10))

In [ ]:
#| export
def conv(ni, nf, kernel_size=3, stride=2, act=nn.ReLU, norm=None, bias=None):
    if bias is None: bias = not isinstance(norm, (nn.BatchNorm1d,nn.BatchNorm2d,nn.BatchNorm3d))
    layers = [nn.Conv2d(ni, nf, stride=stride, kernel_size=kernel_size, padding=kernel_size//2, bias=bias)]
    if norm: layers.append(norm(nf))
    if act: layers.append(act())
    return nn.Sequential(*layers)

def get_model_conv(act=nn.ReLU, nfs=None, norm=None):
    if nfs is None: nfs = [1,8,16,32,64]
    layers = [conv(nfs[i], nfs[i+1], act=act, norm=norm) for i in range(len(nfs)-1)]
    return nn.Sequential(*layers, conv(nfs[-1],10, act=None, norm=False, bias=True),
                         nn.Flatten()).to(def_device)

In [ ]:
model = get_model_conv()
fc.test_eq(model(fc.first(dls.train)[0].to(def_device)).shape,(64,10))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()